In [ ]:
import pandas as pd
import sys
import os

# Set dir
thisfile_dir = os.path.dirname(os.path.abspath(__file__))
dir_cur = os.path.join(thisfile_dir, '..','outputs','extracted_data')
print("Target Directory:", dir_cur)

xlsx_file = "test.pdf.xlsx"
input_file = os.path.join(f"{dir_cur}", xlsx_file)
data = pd.read_excel(input_file)

# Load summary.xlsx
summary_writer = pd.ExcelWriter('summary.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay')
summary = pd.ExcelFile('summary.xlsx')

In [ ]:
# Create a new column as desired
data['column_name'] = data.apply(lambda x: f"{x['label']} ({x['metric']}) ({x['unit']})", axis=1)

# Pivot the table
summary_data = data.pivot(index='year', columns='column_name', values='value')

# The summary_data DataFrame now contains the restructured information
print(summary_data)


In [ ]:
def append_to_sheet(sheet_name, filter_label):
    sheet_data = pd.read_excel(summary, sheet_name=sheet_name)
    filtered_data = data[data['label'].str.contains(filter_label, case=False)]
    
    # Convert units as needed (e.g., GJ to MWhs)
    for i, row in filtered_data.iterrows():
        if row['unit'] == 'GJ':
            filtered_data.at[i, 'value'] = row['value'] / 3.6
            filtered_data.at[i, 'unit'] = 'MWhs'
    
    # Further processing to align the data with the sheet's existing data structure might be needed here...

    new_rows = filtered_data.drop(columns=['label', 'metric', 'unit']).copy()
    new_rows['Company Name'] = 'input name'  # Add company name
    new_rows = new_rows[['Company Name'] + list(new_rows.columns[:-1])]  # Reorder columns if needed

    # Append to the sheet
    sheet_data = pd.concat([sheet_data, new_rows], ignore_index=True)
    sheet_data.to_excel(summary_writer, sheet_name=sheet_name, index=False)

# Append data to 'E' sheet
append_to_sheet('E', 'Energy Consumption')

# Append data to 'S' sheet
# Use appropriate filter_label like 'Gender Diversity', 'Age-Based Diversity', etc.

# Append data to 'G' sheet
# Use appropriate filter_label like 'Board Composition', 'Ethical Behaviour', etc.

# Save the updated summary
summary_writer.save()
summary_writer.close()

print("Data successfully appended to summary.xlsx")